In [2]:
from rdkit import Chem
from rdkit.Chem import AllChem

prod = "[CH3:17][S:14](=[O:15])(=[O:16])[N:11]1[CH2:10][CH2:9][NH:8][CH2:13][CH2:12]1"
reac = "c1ccc(C[N:8]2[CH2:9][CH2:10][N:11]([S:14](=[O:15])(=[O:16])[CH3:17])[CH2:12][CH2:13]2)cc1"

template = "[C]-[NH;D2;+0]-[C]>>[C]-[N;H0;D3;+0](-[C])-C-c1:c:c:c:c:c:1"

s1=[Chem.MolFromSmiles(x) for x in ('NC','NCC')]
s2=[Chem.MolFromSmiles(x) for x in ('OC=O','OC(=O)C')]
rxn = AllChem.ReactionFromSmarts('[O:2]=[C:1][OH].[N:3]>>[O:2]=[C:1][N:3]')
r = AllChem.EnumerateLibraryFromReaction(rxn,[s2,s1])
[Chem.MolToSmiles(x[0]) for x in list(r)]

['CNC=O', 'CCNC=O', 'CNC(C)=O', 'CCNC(C)=O']

In [1]:
from rdkit import Chem
from rdkit.Chem import Draw

# 定义待处理的SMILES字符串
smiles = "Clc1cnc2nc1NCCc1cccc(c1)OCCCc1cccc(c1)N2"

# 将SMILES字符串转换为RDKit的分子对象
mol = Chem.MolFromSmiles(smiles)


# 定义待移除的子结构的SMILES字符串
substructure_smiles = "c1ccccc1"  # 在这里使用您想要去除的子结构的SMILES

# 将待移除的子结构转换为RDKit的分子对象
substructure = Chem.MolFromSmiles(substructure_smiles)

# 使用RDKit进行子结构匹配和去除
matches = mol.GetSubstructMatches(substructure)
for match in matches:
    print(match)
    # 移除匹配到的子结构
    mol3 = Chem.DeleteSubstructs(mol, substructure)

# 打印处理后的分子的SMILES字符串
modified_smiles = Chem.MolToSmiles(mol2)
print("Modified SMILES:", modified_smiles)

# 可选：显示处理后的分子结构
modified_mol = Chem.MolFromSmiles(modified_smiles)
# Draw.MolToImage(modified_mol)

# mol = Chem.MolFromSmiles(smiles)

mol_list = [modified_mol, mol, substructure]
image = Draw.MolsToGridImage(mol_list)
image
# Draw.MolToImage(mol)

(10, 11, 12, 13, 14, 15)
(20, 21, 22, 23, 24, 25)


NameError: name 'mol2' is not defined